# 🏭 ImgCraft Chapter Processor - Lightning AI Studio Runner
## Setup and Run
1. Tải notebook này lên một vùng làm việc trống của Lightning AI Studio.
2. Chạy bước 1 để tải model, clone project và cài đặt môi trường.
3. Chạy bước 2 để khởi động Server và nhận public link qua Pinggy.

In [ ]:
# @title 1. Cài đặt Môi trường & Download Models cho Lightning AI Studio
import os
import subprocess

def run(cmd): 
    print(f"🖥️ Running: {cmd}")
    try:
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"⚠️ Error on command: {cmd}")
        print(f"⚠️ Exit status: {e.returncode}")

print("📦 System Dependencies...")
run("sudo apt-get update && sudo apt-get install -y aria2 libgl1-mesa-glx build-essential")

STUDIO_DIR = "/teamspace/studios/this_studio"
os.chdir(STUDIO_DIR)

# Clone Repo Aniga Pipeline (Chứa mã nguồn imgcraft)
if not os.path.exists('Aniga-Pipeline'):
    run("git clone https://github.com/jofix2004/Aniga-Pipeline.git")
else:
    os.chdir('Aniga-Pipeline')
    run("git fetch origin master")
    run("git reset --hard origin/master")
    os.chdir(STUDIO_DIR)

# Clone ComfyUI
if not os.path.exists(f"{STUDIO_DIR}/ComfyUI"):
    run("git clone https://github.com/jofix2004/ComfyUI.git")

print("📥 Python Requirements...")
run("pip uninstall -y comfy-cli comfy")
run(f"pip install -q -r {STUDIO_DIR}/Aniga-Pipeline/aniga_imgcraft_lightning/requirements.txt")

print("⬇️ Models...")
models = {
    "unet": [("flux1-kontext-dev-fp8-e4m3fn.safetensors", "https://huggingface.co/6chan/flux1-kontext-dev-fp8/resolve/main/flux1-kontext-dev-fp8-e4m3fn.safetensors")],
    "vae": [("ae.sft", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/ae.sft")],
    "clip": [("clip_l.safetensors", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/clip_l.safetensors"), 
             ("t5xxl_fp8_e4m3fn.safetensors", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/t5xxl_fp8_e4m3fn.safetensors")],
    "loras": [("flux_1_turbo_alpha.safetensors", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/flux_1_turbo_alpha.safetensors"),
              ("AniGaKontext_1024x64x1605_v4_000001400.safetensors", "https://huggingface.co/TranLinh2004/AniGaKontext2080v3_Full/resolve/main/AniGaKontext_1024x64x1605_v4_000001400.safetensors")]
}

for folder, items in models.items():
    dest = f"{STUDIO_DIR}/ComfyUI/models/{folder}"
    os.makedirs(dest, exist_ok=True)
    for fname, url in items:
        if not os.path.exists(f"{dest}/{fname}"):
            run(f"aria2c -c -x 16 -s 16 -k 1M -d {dest} -o {fname} {url}")

print("✅ Môi trường & Models đã sẵn sàng!")


In [ ]:
# @title 2. Khởi động ImgCraft Server & Pinggy Tunnel
import threading
import time
import subprocess
import sys
import os
import re
import gc
import torch
import importlib
import uvicorn
import nest_asyncio

nest_asyncio.apply()

STUDIO_DIR = "/teamspace/studios/this_studio"
IMG_CRAFT_DIR = f"{STUDIO_DIR}/Aniga-Pipeline/aniga_imgcraft_lightning"

os.chdir(IMG_CRAFT_DIR)
if IMG_CRAFT_DIR not in sys.path:
    sys.path.insert(0, IMG_CRAFT_DIR)

# ========== NUCLEAR VRAM CLEANUP ==========
print("🧹 [Step 1] Giải phóng VRAM...")

_old_fp = None
if 'imgcraft_server' in sys.modules:
    _old_fp = getattr(sys.modules['imgcraft_server'].state, 'flux_processor', None)

if _old_fp is not None:
    try:
        _old_fp.cleanup()
    except:
        pass
    del _old_fp

try:
    import comfy.model_management as _mm
    _mm.unload_all_models()
    _mm.soft_empty_cache()
except: pass

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

# ========== RELOAD MODULES ==========
print("🔄 [Step 2] Reload modules...")
import imgcraft_core
import imgcraft_server
importlib.reload(imgcraft_core)
importlib.reload(imgcraft_server)

# ========== KILL TIẾN TRÌNH CŨ ==========
print("🔪 [Step 3] Kill tiến trình cũ...")
subprocess.run("fuser -k 8001/tcp", shell=True, stderr=subprocess.DEVNULL)
subprocess.run("pkill -f uvicorn", shell=True)
subprocess.run("pkill -f 'ssh.*pinggy'", shell=True)
time.sleep(2)

# ========== INIT MỚI ==========
print("📦 [Step 4] Khởi tạo FluxProcessor...")
imgcraft_server.state.flux_processor = imgcraft_core.FluxProcessor()

print("🚀 [Step 5] Bật Server (Port 8001)...")
server_config = uvicorn.Config(imgcraft_server.app, host="0.0.0.0", port=8001, log_level="error")
server = uvicorn.Server(server_config)
t = threading.Thread(target=server.run, daemon=True)
t.start()
time.sleep(5)

print("🌐 [Step 6] Bật Pinggy Tunnel...")
cmd = "ssh -p 443 -o StrictHostKeyChecking=no -R0:localhost:8001 a.pinggy.io"
process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

print("="*50)
print("👇👇👇 LINK TRUY CẬP IMGCRAFT: 👇👇👇")
try:
    url_printed = False
    for line in iter(process.stdout.readline, ''):
        if not url_printed and "https://" in line:
            if "dashboard.pinggy.io" in line:
                continue
            match = re.search(r"https://[a-zA-Z0-9._-]+\.a\.free\.pinggy\.link", line)
            if not match:
                match = re.search(r"https://[a-zA-Z0-9._-]+\.pinggy\.[a-z]+", line)
            if match:
                print(f"\n🔗 URL: {match.group(0)}\n")
                print("="*50)
                url_printed = True
except KeyboardInterrupt:
    process.kill()
    print("\n🛑 Tunnel stopped.")
